In [ ]:
# !pip install transformers

In [1]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import warnings

warnings.filterwarnings('ignore')


In [ ]:
origin_df = pd.read_csv('C:/Users/Lee_Hyo_Jae/Desktop/new_project/dataset/intent_model_dataset.csv')
df = copy.deepcopy(origin_df)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.dropna(inplace=True)
df.drop(df[df.duplicated()].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(언약) 위협하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 부정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 긍정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(선언/위임하기)'].index, axis=0, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# 클래스 개수
lable_num = len(df['intent'].unique())
lable_num

In [ ]:
# str -> int 라벨변경
intet_label = list(df['intent'].unique())

label_dict = {}

for idx, intent_lab in enumerate(intet_label) :
    df.loc[df['intent'] == intent_lab, 'intent'] = idx
    label_dict[idx] = intent_lab

print(label_dict)
df

In [ ]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()
print(train_data)
print(test_data)

for k,v in label_dict.items():
    len_ = df.loc[df['intent'] == k,'intent'].count()
    if len_ > 20000 :
        temp = df.loc[df['intent'] == k].sample(n=20000).reset_index(drop=True)
        train_data = pd.concat([train_data,temp.loc[:16000]],ignore_index=True)
        test_data = pd.concat([test_data,temp.loc[16000:]],ignore_index=True)
    else :
        temp = df.loc[df['intent'] == k].reset_index(drop=True)
        train_data = pd.concat([train_data,temp.loc[:(len_*8)/10]],ignore_index=True)
        test_data = pd.concat([test_data,temp.loc[(len_*8)/10:]],ignore_index=True)

train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)


In [38]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [39]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

In [40]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True, truncation=True)

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=128, validation_split=0.2)

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=64)
print("test loss, test acc: ", results)

In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])
    
    print('sentence :',new_sentence)
    print(label_dict[score])


In [ ]:
MODEL_SAVE_PATH = os.path.join("intent_classifier")
print(MODEL_SAVE_PATH)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)